In [8]:
import os
import zipfile
import re

def extract_fMRI_atlas(input_path, output_path, field_id, filename_in_zip, atlas_name):
    """
    Extract a specific fMRI atlas time series from UK Biobank ZIP files.

    Parameters:
        input_path (str): Path to folder containing ZIPs (e.g., '52/')
        output_path (str): Path to save extracted .csv.gz files
        field_id (str): UKB field ID (e.g., '31016' for Glasser)
        filename_in_zip (str): Name of file inside ZIP (e.g., 'fMRI.Glasser.csv.gz')
        atlas_name (str): Short name for output (e.g., 'Glasser')
    """
    os.makedirs(output_path, exist_ok=True)

    for root, dirs, files in os.walk(input_path):
        for file in files:
            if not file.endswith('.zip'):
                continue
            match = re.match(r'(\d+)_(\d+)_(\d+)_(\d+)\.zip', file)
            if not match:
                continue

            subject_id, fid, run_id, version = match.groups()
            if fid != field_id:
                continue

            zip_path = os.path.join(root, file)
            try:
                with zipfile.ZipFile(zip_path, 'r') as z:
                    if filename_in_zip in z.namelist():
                        out_name = f"sub-{subject_id}_run-{run_id}_{atlas_name}.csv.gz"
                        out_path = os.path.join(output_path, out_name)
                        with z.open(filename_in_zip) as src, open(out_path, 'wb') as dst:
                            dst.write(src.read())
            except Exception as e:
                print(f"[ERROR] {file}: {e}")

    print(f"✅ Done. {atlas_name} files saved to: {output_path}")

In [9]:
extract_fMRI_atlas(
    input_path='52',
    output_path='atlas',
    field_id='31016',
    filename_in_zip='fMRI.Glasser.csv.gz',
    atlas_name='Glasser'
)

✅ Done. Glasser files saved to: atlas


In [11]:
extract_fMRI_atlas(
    input_path='52',
    output_path='atlas',
    field_id='31019',
    filename_in_zip='fMRI.Tian_Subcortex_S4_3T.csv.gz',
    atlas_name='TianS4'
)

✅ Done. TianS4 files saved to: atlas


In [16]:
import os
import pandas as pd

# === CONFIG ===
atlas_path = 'atlas'
subject_id = "5200118"   # ← CHANGE THIS
run_id = "2"             # ← CHANGE THIS

# === FILENAMES ===
glasser_file = f"sub-{subject_id}_run-{run_id}_Glasser.csv.gz"
tian_file    = f"sub-{subject_id}_run-{run_id}_TianS4.csv.gz"

glasser_path = os.path.join(atlas_path, glasser_file)
tian_path    = os.path.join(atlas_path, tian_file)

# === CHECK FILES EXIST ===
if not os.path.exists(glasser_path):
    raise FileNotFoundError(f"Glasser file not found: {glasser_file}")
if not os.path.exists(tian_path):
    raise FileNotFoundError(f"TianS4 file not found: {tian_file}")

# === LOAD ===
print("Loading Glasser...")
glasser_ts = pd.read_csv(glasser_path, compression='gzip')
print("Loading TianS4...")
tian_ts = pd.read_csv(tian_path, compression='gzip')

# === VERIFY ALIGNMENT ===
print(f"Glasser shape: {glasser_ts.shape}")   # (timepoints, regions)
print(f"TianS4 shape:  {tian_ts.shape}")




Loading Glasser...
Loading TianS4...
Glasser shape: (360, 491)
TianS4 shape:  (54, 491)


In [17]:
# Transpose: (regions, time) → (time, regions)
glasser_ts_T = glasser_ts.set_index(glasser_ts.columns[0]).T  # assumes col 0 = region names
tian_ts_T    = tian_ts.set_index(tian_ts.columns[0]).T

print("Transposed shapes:")
print("Glasser:", glasser_ts_T.shape)   # (491, 360)
print("TianS4:", tian_ts_T.shape)      # (491, 54)

Transposed shapes:
Glasser: (490, 360)
TianS4: (490, 54)


In [18]:
glasser_ts_T

label_name,R_V1_ROI,R_MST_ROI,R_V6_ROI,R_V2_ROI,R_V3_ROI,R_V4_ROI,R_V8_ROI,R_4_ROI,R_3b_ROI,R_FEF_ROI,...,L_p47r_ROI,L_TGv_ROI,L_MBelt_ROI,L_LBelt_ROI,L_A4_ROI,L_STSva_ROI,L_TE1m_ROI,L_PI_ROI,L_a32pr_ROI,L_p24_ROI
timepoint_0,14534.040215,20098.204881,15520.598185,14951.072767,19244.875308,16294.674207,10439.459706,11553.672179,14879.606674,13634.538954,...,14301.778226,4211.228788,9143.778984,9030.816485,14695.856242,7259.926646,6325.548247,8795.976206,8001.729150,8188.778265
timepoint_1,14455.935916,20167.841760,15479.647542,14947.504478,19238.108253,16265.498063,10443.994233,11557.279696,14836.775654,13621.441699,...,14319.278295,4182.030943,9016.917407,9044.897870,14713.991880,7272.397915,6288.087665,8715.806865,7978.779730,8224.919713
timepoint_2,14478.879372,20141.459801,15477.767456,14940.424301,19252.426785,16230.775106,10344.580245,11550.816248,14843.888492,13588.076584,...,14231.086109,4176.914162,9047.990603,9080.891536,14675.354010,7259.663999,6308.165485,8727.945049,7923.114334,8161.526381
timepoint_3,14453.269593,20100.909595,15510.856812,14947.633415,19184.785008,16193.963335,10442.767704,11567.071158,14780.554764,13625.278505,...,14304.016199,4148.079059,9085.203655,8980.698612,14726.465005,7293.647803,6315.957139,8773.607376,7901.623972,8164.228460
timepoint_4,14523.564963,20129.347139,15498.337972,14974.580468,19269.855948,16296.661948,10494.122951,11561.201206,14812.303406,13670.118468,...,14338.644060,4183.443648,9079.537330,9051.439031,14728.370358,7282.189780,6310.766456,8774.196771,7944.436534,8227.005329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
timepoint_485,14491.263782,20163.256063,15481.298001,14930.166784,19233.636971,16241.404225,10416.857969,11570.443859,14858.938227,13675.128353,...,14392.008215,4186.116624,9105.782853,9070.804595,14711.585332,7264.694106,6309.391901,8773.140915,7960.753760,8178.196233
timepoint_486,14530.518549,20162.319620,15474.003852,14968.743472,19274.872173,16285.912197,10434.243331,11574.266421,14836.531769,13663.780469,...,14344.054533,4187.206590,9134.799362,9037.793074,14741.074352,7250.207749,6322.910722,8717.957553,7959.896758,8184.441616
timepoint_487,14549.709561,20146.960580,15472.530450,14928.046384,19263.982208,16245.531428,10417.087672,11555.503917,14809.870948,13716.948969,...,14430.863581,4193.027042,9142.775188,9030.121595,14656.687915,7248.047153,6300.421183,8710.907451,7910.656716,8115.127846
timepoint_488,14414.877975,20118.001654,15416.656779,14880.725331,19183.067183,16207.373201,10418.991196,11556.020144,14813.602047,13690.531977,...,14372.754729,4186.642674,9095.588701,8994.945986,14726.959026,7253.034487,6295.369851,8710.530636,7978.120452,8175.550949


In [19]:
tian_ts_T

label_name,HIP-head-m1-rh,HIP-head-m2-rh,THA-VAip-rh,THA-VAia-rh,HIP-head-l-rh,HIP-body-rh,HIP-tail-rh,THA-VPm-rh,THA-VPl-rh,THA-VAs-rh,...,CAU-DA-lh,CAU-body-lh,CAU-tail-lh,lAMY-lh,mAMY-lh,THA-DP-lh,NAc-shell-lh,NAc-core-lh,pGP-lh,aGP-lh
timepoint_0,4205.164662,4434.532084,5900.340835,5801.017004,5836.480183,7411.051066,8415.087737,6225.459979,5943.122057,5447.801280,...,6550.831250,6214.953835,6135.495072,3894.972947,2924.371922,6165.098790,126.279015,2509.632161,3127.800000,1246.106909
timepoint_1,4164.975398,4497.495422,5848.985227,5819.959631,5814.843974,7329.927550,8376.147001,6211.690992,5967.473543,5447.556698,...,6472.847789,6203.965687,6018.375416,3878.025433,2945.291158,6118.957517,127.891197,2448.654418,3112.422806,1190.697326
timepoint_2,4264.660247,4518.672496,5874.595009,5845.737850,5799.051366,7386.226837,8423.473158,6230.431244,5929.244218,5480.293142,...,6418.825701,6192.579878,6110.800438,3874.438008,2949.979686,6106.488872,155.303657,2507.187312,3016.557440,1174.854835
timepoint_3,4198.721505,4562.944234,5844.933468,5771.247854,5844.006412,7344.505758,8472.941596,6215.966115,5918.000398,5451.550682,...,6477.591269,6137.781060,6071.843171,3888.184530,2929.624340,6104.015842,135.222060,2453.208999,3117.213763,1187.776579
timepoint_4,4310.157471,4417.328389,5843.921338,5786.454982,5837.475615,7331.403659,8371.523379,6252.628860,5966.401676,5406.714368,...,6425.819620,6115.518884,6067.490533,3863.970531,2939.912684,6084.060441,129.252973,2478.736530,3080.929124,1223.012843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
timepoint_485,4214.722245,4484.351471,5889.696516,5839.894497,5819.235673,7383.466844,8426.269462,6225.745090,5946.774722,5460.267444,...,6507.630669,6183.350169,6093.041233,3883.467347,2940.303448,6159.047782,128.825576,2472.974739,3105.129465,1217.227301
timepoint_486,4227.076151,4454.414988,5849.918194,5833.587009,5839.125848,7348.769127,8406.610998,6249.802569,5978.358553,5498.391197,...,6499.174316,6210.230419,6047.528474,3878.093864,2940.395829,6138.227217,136.839743,2476.031291,3107.494403,1215.915691
timepoint_487,4202.085156,4503.770325,5854.219749,5806.025174,5812.990119,7368.845883,8434.977068,6216.170926,5950.293907,5448.543759,...,6463.957660,6163.121789,6081.191741,3886.812343,2908.406677,6094.062156,140.024955,2498.649936,3101.964545,1201.873458
timepoint_488,4262.494510,4464.524841,5879.288685,5790.329267,5831.962133,7376.775768,8413.804396,6217.491635,5963.232807,5445.454129,...,6472.805479,6194.778496,6125.938929,3894.679282,2944.795879,6103.729251,129.328325,2461.404247,3136.714316,1223.729000
